In [ ]:
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.applications.resnet50 import preprocess_input
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score, cohen_kappa_score, accuracy_score
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import os
import pandas as pd
import numpy as np
from tensorflow.python.keras.utils.data_utils import Sequence
import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.listdir('/kaggle/input/predict-volcanic-eruptions-ingv-oe')

In [ ]:
train_df = pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train.csv')
train_df.head()

In [ ]:
filename = []
for file in train_df['segment_id']:
    filename.append('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/'+str(file)+'.csv')
train_df['filename'] = filename
train_df = train_df.drop('segment_id',axis = 1)
train_df.head()

In [ ]:
test_df = pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')
test_df.head()

In [ ]:
filename = []
for file in test_df['segment_id']:
    filename.append('/kaggle/input/predict-volcanic-eruptions-ingv-oe/test/'+str(file)+'.csv')
test_df['filename'] = filename
test_df.head()

In [ ]:
x = train_df['filename']
y = train_df['time_to_eruption']

train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.2, random_state=8)
print(train_x.shape)
print(train_y.shape)
print(valid_x.shape)
print(valid_y.shape)

In [ ]:
train = []
for samp,lab in zip(train_x,train_y):
    train.append([samp,lab])
valid = []
for samp,lab in zip(valid_x,valid_y):
    valid.append([samp,lab])

In [ ]:
def generator(samples, batch_size=32,shuffle=True):
    num_samples = len(samples)
    while True: 
        np.random.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            x_train = []
            y_train = []
            for batch_sample in batch_samples:
                filename = batch_sample[0]
                label = batch_sample[1]
                df =  pd.read_csv(filename).fillna(0)
                x_train.append(df.values)
                y_train.append(label)
            x_train = np.array(x_train)
            y_train = np.array(y_train)          
            yield x_train, y_train

In [ ]:
train_datagen = generator(train,batch_size=8,shuffle=True)

x,y = next(train_datagen)
print (x.shape)
#output: (8, 224, 224, 3)
print (y)
#output: [0 1 1 4 3 1 4 2]

In [ ]:
n_timesteps, n_features, n_outputs = 60001, 10, 1

In [ ]:
train_generator = generator(train, batch_size=64)
validation_generator = generator(valid, batch_size=64)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D, Conv1D, MaxPooling1D)
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs))

In [ ]:
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger)

epochs = 200

checkpoint = ModelCheckpoint('../working/conv1d_volcanic.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, 
                                   verbose=1, mode='min', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=20)

In [ ]:
model.summary()

In [ ]:
num_train_samples = train_x.shape[0]
num_valid_samples = valid_x.shape[0]

In [ ]:
callbacks_list = [checkpoint, reduceLROnPlat, early]
model.compile(loss='mae', optimizer='rmsprop', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.fit_generator(
        train_generator,
        steps_per_epoch=num_train_samples // 64,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=num_valid_samples // 64,
        callbacks=callbacks_list)